In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data = pd.read_csv("../input/time-series-starter-dataset/Month_Value_1.csv")

In [ ]:
data.head()

In [ ]:
plt.figure(figsize = (18,10))
plt.plot(data['Period'],data['Revenue'])

In [ ]:
data.info()

In [ ]:
train = data.iloc[:64]
test = data.iloc[64:]

In [ ]:
test

In [ ]:
train.drop('Period', axis = 1, inplace = True)

In [ ]:
from sklearn.preprocessing import MinMaxScaler


In [ ]:
scaler = MinMaxScaler()

In [ ]:
scaled_train = scaler.fit_transform(train)

In [ ]:
scaled_train

In [ ]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
plt.figure(figsize = (18,10))
plt.plot(train['Revenue'])

In [ ]:
length = 12

batch_size = 1

generator = TimeseriesGenerator(scaled_train,scaled_train,batch_size=batch_size, length = length)

In [ ]:
len(scaled_train)

In [ ]:
len(generator)

In [ ]:
from tensorflow.keras.models import Sequential

In [ ]:
from tensorflow.keras.layers import Dense,SimpleRNN,LSTM

In [ ]:
n_features = 4

In [ ]:
model = Sequential()

model.add(SimpleRNN(50,input_shape = (length,n_features)))
model.add(Dense(4))

model.compile(loss = 'mse', optimizer = 'adam')

In [ ]:
model.fit_generator(generator,epochs=50)

In [ ]:
losses = pd.DataFrame(model.history.history)

In [ ]:
losses.plot()

In [ ]:
first_eval_batch = scaled_train[-length:]

In [ ]:
first_eval_batch = first_eval_batch.reshape(1,length,n_features)

In [ ]:
model.predict(first_eval_batch)

In [ ]:
scaled_train[13]

In [ ]:
prediction = []

first_eval_batch = scaled_train[-length:]
current_batch = first_eval_batch.reshape(1,length,n_features)

for i in range(len(test)):
    current_pred = model.predict(current_batch)[0]
    prediction.append(current_pred)
    current_batch = np.append(current_batch[:,1:,:], [[current_pred]], axis = 1)

In [ ]:
prediction

In [ ]:
true_prediction = scaler.inverse_transform(prediction)

In [ ]:
len(true_prediction)

In [ ]:
predicted = pd.DataFrame(true_prediction, columns=train.columns)

In [ ]:
predicted_index = np.arange(65,97,step =1)

In [ ]:
plt.plot(train.index,train['Revenue'])
plt.plot(predicted_index,predicted['Revenue'])